In [ ]:
import distutils.util
import subprocess
#if subprocess.run('nvidia-smi').returncode:
#  raise RuntimeError(
#      'Cannot communicate with GPU. '
#      'Make sure you are using a GPU Colab runtime. '
#      'Go to the Runtime menu and select Choose runtime type.')

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

import mujoco

#@title Other imports and helper functions
import numpy as np

# Graphics and plotting.
import mediapy as media
import matplotlib.pyplot as plt

# Import Renderer Class
import os
import sys

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
from renderer import Renderer

import platform
print (platform.architecture())

### Load Model

In [ ]:
model = mujoco.MjModel.from_xml_path('modelPark.xml')
data = mujoco.MjData(model)
renderer = Renderer(model)

### Example Video

Generate a short example video to check that the model and camera are set up correctly.

In [ ]:
# Parameters about video generation
DURATION  = 1   # seconds
FRAMERATE = 30  # Hz
frames = []

# A camera named side is predefined in the model, 
# and here we use this camera to generate video.
camera_side = model.camera('side').id

while data.time < DURATION:
  # Step the simulation.
  mujoco.mj_step(model, data)

  # Render and save frames.
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data, camera_side)
    pixels = renderer.render()
    frames.append(pixels.copy())

# Display video.
media.show_video(frames, fps=FRAMERATE)

### PID control

Suppose we want to maintain the inclination angle between the segway and the ground to $α_0$
The current time is $t$, the inclination of the segway is α_t.
Calculate the error:
$$
{err}_t = α_t - α_0 
$$
The controllable variable is the torque acting on the two wheels, that is, the ${qval}_t$ of the wheel in the majoco model（about time $t$).
Then there is the following formula：
$$
{qval}_t = K_p∙{err}_t + K_i∙\sum_{n=t-t_0}^t {err}_n + K_d∙({err}_t-{err}_{t-1})
$$
$K_p$$∙$${err}_t$ is the $P$ item, namely proportion, the error ${err}_t$ is multiplied by a certain constant $K_p$.

$K_i$$∙$$\sum_{n=t-t_0}^t$${err}_n$ is the $I$ item, that is, integration, the sum of the error ${err}_t$(because the time in majoco is discrete, it is not an integral but a summation) multiplied by a certain constant $K_I$。

$K_d∙({err_t}-{err}_{t-1})$ is the $D$ item, that is, differentiation, the change of error err with respect to time $t$, multiplied by a certain constant $K_d$。

The value of $K_p, K_i, K_d$ is determined by observation and experiment.
In particular, we use $K_i = 0$ in this project because the physical environment of the simulated environment is ideal and there is not much noise.

#### helper functions

Two arrays are defined to record the current angle and target angle, and these arrays will be used to generate graphs to help evaluate the performance of PID control.

In [ ]:
current_angle_4plot = np.array([])
target_angle_4plot = np.array([])

In [ ]:
# Calculates the angle between two vectors in 3D space.
def get_angle(v1,v2):
    x=np.array(v1)
    y=np.array(v2)
    module_x=np.sqrt(x.dot(x))
    module_y=np.sqrt(y.dot(y))
    dot_value=x.dot(y)
    cos_theta=dot_value/(module_x*module_y)
    angle_radian=np.arccos(cos_theta)
    angle_value=angle_radian*180/np.pi
    return angle_value

# Extract the required data(angle and velocity) according to the coordinate information in the model. 
# In practice, these data are provided by internal gyroscopes and accelerometers.
def get_state(old_coordinate, new_coordinate):
    
    # There are coordinate data of the previous moment and the current moment. 
    new_refpoint1 = new_coordinate[0]
    new_refpoint2 = new_coordinate[1]
    new_refpoint3 = new_coordinate[2]
    new_refpoint4 = new_coordinate[3]
    old_refpoint1 = old_coordinate[0]
    old_refpoint2 = old_coordinate[1]
    old_refpoint3 = old_coordinate[2]
    old_refpoint4 = old_coordinate[3]
    
    old_vector_pole = old_refpoint2 - old_refpoint1
    old_mid_point = (old_refpoint3 + old_refpoint4) / 2
    
    new_vector_pole = new_refpoint2 - new_refpoint1
    new_mid_point = (new_refpoint3 + new_refpoint4) / 2
        
    delta_mid_point = new_mid_point - old_mid_point
    delta_mid_point_2d = np.array([delta_mid_point[0], delta_mid_point[1]], dtype='float32')
    
    new_dir_vector = new_refpoint1 - new_mid_point
    new_dir_vector_2d = np.array([new_dir_vector[0], new_dir_vector[1]], dtype='float32')
    
    old_dir_vector = old_refpoint1 - old_mid_point
    old_dir_vector_2d = np.array([old_dir_vector[0], old_dir_vector[1]], dtype='float32')
    
    # new_angle_pole_dir is the proportion err_t in PID control
    new_angle_pole_dir = get_angle(new_vector_pole, np.array([new_dir_vector[0], new_dir_vector[1], 0], dtype='float32'))
    old_angle_pole_dir = get_angle(old_vector_pole, np.array([old_dir_vector[0], old_dir_vector[1], 0], dtype='float32'))
    
    # delta_angle is the differentiation err_t − err_(t-1) in PID control
    delta_angle = new_angle_pole_dir - old_angle_pole_dir
    speed = delta_mid_point_2d.dot(new_dir_vector_2d)
    wheel_height_difference = abs(new_refpoint3[2] - new_refpoint4[2])
    
    return np.float32(np.hstack((new_angle_pole_dir, delta_angle, speed, wheel_height_difference)))

# Extract the coordinate information in the model. 
def get_coordinate(data):
    # Four coordinate points are located at the two ends of the bearing and 
    # the two ends of the joystick respectively. For specific locations, please refer to the model file.
    refpoint1 = np.array(data.geom_xpos[model.geom("refpoint1").id])
    refpoint2 = np.array(data.geom_xpos[model.geom("refpoint2").id])
    refpoint3 = np.array(data.geom_xpos[model.geom("refpoint3").id])
    refpoint4 = np.array(data.geom_xpos[model.geom("refpoint4").id])
    ans = np.array([refpoint1, refpoint2, refpoint3, refpoint4], np.float32)
    return ans

def toCtrl(state, move, direction, speed, diff):
    # If the height difference between the two wheels is too large, the segway is out of balance.
    # In this case, set the qval of the wheel to 0.
    if(state[3] > 0.12):
        return np.array([0, 0], dtype='float64')
    
    # Through observation and experiment, 86.9° is the static angle of segway.
    tar_angle = 86.9 
    
    # The target angle of the PID depends on 
    # which way(eg. move = 1 for move forward) we want the segway to go 
    # and at which speed.
    tar_angle = tar_angle - speed * move
    
    # If move is 0, apply brakes.
    if(move == 0):
        tar_angle = tar_angle + state[2] * 1200
    
    # The definition of hyperparameters depends on observation and experimentation.
    kd = -0.1
    kp = -1
    
    # PID formula, only PD is used here.
    qval =  kp * state[1] + kd * (state[0] - tar_angle)
    
    # limited maximum torque.
    maxqval = 1
    qval = max(min(qval, maxqval),-maxqval)
    ctrl = np.array([qval, qval], dtype='float64')
    
    # If the segway receives an order to turn, it multiplies the torque on one side of the tire.
    if(move == 1 and direction != 2):
        ctrl[direction] *= diff
    
    # Record current and target angles.
    global current_angle_4plot
    global target_angle_4plot
    current_angle_4plot = np.append(current_angle_4plot, state[0])
    target_angle_4plot = np.append(target_angle_4plot, tar_angle)
    
    return ctrl

### simulation

In [ ]:
# The control method of segway. Defined by numbers.
# move:      forward = 1 backward = -1  hold = 0
# direction: right = 0    left = 1      hold = 2
move = 0
direction = 2
speed = 3
diff = 2

# Initialize the model
mujoco.mj_resetData(model, data)
mujoco.mj_step(model, data)
old_coordinate = get_coordinate(data)
mujoco.mj_step(model, data)
new_coordinate = get_coordinate(data)
state = get_state(old_coordinate, new_coordinate)
frames = []

# start simulation
ep_step = 0
while True:
    ep_step += 1
    
    # Manually Defined Action Sequences
    match ep_step:
        case 50:
            direction = 2
            move = 1
        case 100:
            direction = 1
        case 150:
            direction = 2
        case 700:
            direction = 1
        case 790:
            direction = 2
            speed = -1
        case 1000:
            direction = 0
            speed = 3
        case 1100:
            speed = -1
        case 1600:
            direction = 2
            speed = 3
        case 1750:
            direction = 1
            speed = -1
        case 1900:
            direction = 2
            speed = 1
        case 2200:
            direction = 1
        case 2450:
            diff = 3
            direction = 0
        case 3000:
            direction = 2
            speed = -1
        case 3400:
            speed = 3
    
    # Generate controls for the tires on based on actions.
    data.ctrl = toCtrl(state, move, direction, speed, diff) 
    
    # Repeat an action 6 times.
    # Repetitive actions help to reduce the space for human manipulation and 
    # reduce the difficulty of generating action sequences.
    for x in range(6):
        if len(frames) < data.time * FRAMERATE:
            renderer.update_scene(data, camera_side)
            pixels = renderer.render()
            frames.append(pixels.copy())
        mujoco.mj_step(model, data)
    
    # Collect data for next action.
    old_coordinate = new_coordinate
    new_coordinate = get_coordinate(data)
    state = get_state(old_coordinate, new_coordinate)  
    
    # The length of the simulation is controlled by ep_step.
    if ep_step == 2300:
        break
        
media.show_video(frames, fps=FRAMERATE)

By observing the following graphs, we can better tune the individual hyperparameters in PID.

In [ ]:
plt.xlabel('time')
plt.ylabel('angle')
plt.plot(current_angle_4plot, label='current angle')
plt.plot(target_angle_4plot, label='target angle')
plt.title("Performance of PID Control")
plt.legend()